In [1]:
import cv2
import xml.etree.ElementTree as ET
import numpy as np

In [2]:
imgsrc=cv2.imread('/home/lishuang/Disk/pytorch/test/img/image_2017_04_20_01_29.jpg')

tree = ET.parse('/home/lishuang/Disk/pytorch/test/xml/image_2017_04_20_01_29.xml')
root = tree.getroot()

print(imgsrc.shape)  #H W C
size=root.find('size')
width=int(size.find('width').text)
height=int(size.find('height').text)
objs=root.findall('object')
print("width =",width,"height =",height)
box=[]
for obj in objs:
    name=obj.find('name').text  #type
    bndbox=obj.find('bndbox')
    xmin=int(bndbox.find('xmin').text)
    ymin=int(bndbox.find('ymin').text)
    xmax=int(bndbox.find('xmax').text)
    ymax=int(bndbox.find('ymax').text)
    box.append([xmin,ymin,xmax,ymax])
    print(box)

box=np.array(box)

#show origin image
# for box_id in range(len(box)):
#     cv2.rectangle(imgsrc,(int(box[box_id][0]),int(box[box_id][1])),(int(box[box_id][2]),int(box[box_id][3])),(0,255,0),2)
# cv2.imwrite('imgsrc.jpg',imgsrc)    


(480, 640, 3)
width = 640 height = 480
[[126, 379, 235, 479]]
[[126, 379, 235, 479], [121, 163, 251, 295]]


In [3]:
class data_augmentation:

    def __init__(self,img,box,gamma=1.0,log=1.0):
        self.img=img.copy()
        self.box=box.copy()
        gamma_table = [np.power(x, gamma)  for x in range(256)]
        gamma_table = np.array(gamma_table)
        self.gamma_table=gamma_table
        log_table = [np.log(1 + x) * log for x in range(256)]
        log_table = np.array(log_table)
        self.log_table = log_table

    def resize(self,minscale=0.6,maxscale=1.4,step=0.1):
        height, width = self.img.shape[0], self.img.shape[1]
        scale=np.random.choice(np.arange(minscale, maxscale, step))
        resize_width=width*scale
        resize_height=height*scale
        resize_box=self.box*scale
        resize_img=cv2.resize(self.img,(int(resize_width),int(resize_height)))
        
        return resize_img,resize_box.astype('int64')

    def crop(self,jitter=0.3,percent=0.5):#percent=dstbox/srcbox
        height, width = self.img.shape[0], self.img.shape[1]
        dw=width*jitter
        dh=height*jitter
        pleft=np.random.randint(-dw,dw+1)
        pright=np.random.randint(-dw,dw+1)
        ptop=np.random.randint(-dh,dh+1)
        pbot=np.random.randint(-dh,dh+1)
        swidth =  width - pleft - pright
        sheight = height - ptop - pbot
        
        imgdst=np.zeros((sheight,swidth,self.img.shape[2]))
        
        crop_img=self.img[max(0,ptop):min(height,height-pbot),max(0,pleft):min(width,width-pright),:]
        crop_img_h=crop_img.shape[0]
        crop_img_W=crop_img.shape[1]
        xmin=abs(min(0,pleft))
        ymin=abs(min(0,ptop))
        imgdst[ymin:ymin+crop_img_h,xmin:xmin+crop_img_W,:]=crop_img
        #self.img=imgdst.copy()
        
        crop_img_xmin=max(0,pleft)
        crop_img_ymin=max(0,ptop)
        crop_img_xmax=min(width,width-pright)
        crop_img_ymax=min(height,height-pbot)
        
        xchange=xmin-crop_img_xmin
        ychange=ymin-crop_img_ymin
        boxnew=[]
        for box_id in range(len(self.box)):
            boxtmp=self.box[box_id]
            xmintmp=boxtmp[0]
            ymintmp=boxtmp[1]
            xmaxtmp=boxtmp[2]
            ymaxtmp=boxtmp[3]
            xminnew=max(xmintmp,crop_img_xmin)
            yminnew=max(ymintmp,crop_img_ymin)
            xmaxnew=min(xmaxtmp,crop_img_xmax)
            ymaxnew=min(ymaxtmp,crop_img_ymax)
            if xmaxnew>xminnew and ymaxnew>yminnew:
                squarenew=(xmaxnew-xminnew)*(ymaxnew-yminnew)
                squaretmp=(xmaxtmp-xmintmp)*(ymaxtmp-ymintmp)
                if squarenew > percent*squaretmp:
                    #change old coordinate to new coordinate
                    boxnew.append([xminnew+xchange,yminnew+ychange,xmaxnew+xchange,ymaxnew+ychange])
        
        boxnew=np.array(boxnew)
                
            
        
        return imgdst,boxnew
        
    def lr_filp(self):
        height, width = self.img.shape[0], self.img.shape[1]
        lr_img=np.fliplr(self.img)
        lr_box=[]
        for box_id in range(len(self.box)):
            xmax = width-1-self.box[box_id][0]
            xmin = width-1-self.box[box_id][2]
            ymin=self.box[box_id][1]
            ymax=self.box[box_id][3]
            lr_box.append([xmin,ymin,xmax,ymax])
            #lr_box[box_id]=[xmin,ymin,xmax,ymax]
            
        lr_box=np.array(lr_box)
        return lr_img,lr_box
        
    def up_filp(self):
        height, width = self.img.shape[0], self.img.shape[1]
        up_img = np.flipud(self.img)
        up_box=[]
        for box_id in range(len(self.box)):
            
#             boxtmp=self.box[box_id]
#             ymax=height-1-boxtmp[1]
#             ymin=height-1-boxtmp[3]
#             boxtmp[1]=ymin
#             boxtmp[3]=ymax
#             up_box.append(boxtmp)
            
            ymax=height-1-self.box[box_id][1]
            ymin=height-1-self.box[box_id][3]
            xmin=self.box[box_id][0]
            xmax=self.box[box_id][2]
            up_box.append([xmin,ymin,xmax,ymax])
            
        up_box=np.array(up_box)
        return up_img,up_box
    
    def hsv(self,hue=0.1,saturation=1.5,exposure=1.5):
        dhue=np.random.uniform(-hue,hue)
        if np.random.rand() >0.5:
            dsat=np.random.uniform(1,saturation)
        else:
            dsat=1.0/np.random.uniform(1,saturation)
        
        if np.random.rand() >0.5:
            dexp=np.random.uniform(1,exposure)
        else:
            dexp=1.0/np.random.uniform(1,exposure)
            
        img_hsv=cv2.cvtColor(self.img,cv2.COLOR_BGR2HSV)
        (H, S, V) = cv2.split(img_hsv)
        H=H.astype(np.float32)
        S=S.astype(np.float32)
        V=V.astype(np.float32)
        
        S*=dsat
        V*=dexp
        H+=179*dhue
        
        H=H.astype(np.uint8)
        S=S.astype(np.uint8)
        V=V.astype(np.uint8)
        
        
        img_hsv=cv2.merge((H, S, V))
        img_hsv=cv2.cvtColor(img_hsv,cv2.COLOR_HSV2BGR)
        
        return img_hsv,self.box
    
    def mix_up(self,imgsrc,boxsrc,alpha=0.5,beta=0.5,gamma=0):#base come from yolo
        for i in range(len(self.img.shape)):
            assert(self.img.shape[i]==imgsrc.shape[i])
        imgdst=cv2.addWeighted(self.img,alpha,imgsrc,beta,gamma)
        
        boxnew=np.concatenate((box,boxsrc))
        return imgdst,boxnew
    
    def blur(self,kw=7,kh=7,sigmaX=0,sigmaY=0):
        #opencv 
        if sigmaY==0:
            sigmaY=sigmaX
        dst=cv2.GaussianBlur(self.img,(kw,kh),sigmaX,sigmaY)
#         dst=cv2.blur(self.img,(5,5))
#         dst=cv2.bilateralFilter(self.img,9,75,75)
#         dst=cv2.medianBlur(img,5)
        return dst,self.box
    
    def frosted_glass(self,mm=3):
        height=self.img.shape[0]
        width=self.img.shape[1]
        #dst=np.zeros([height,width,3],np.uint8)
        dst=self.img.copy()
        for m in range(height-mm):
            for n in range(width-mm):
                index=int(np.random.random()*mm)
                (b,g,r)=self.img[m+index,n+index]
                dst[m,n]=(b,g,r)

        return dst,self.box
    #from skimage.util import random_noise also apply noise function
    def guass_noisy(self,mean=0,var=0.1):
        image=self.img
        row,col,ch= image.shape
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + image*gauss
        return noisy,self.box
    
    def sp_noisy(self,s_vs_p=0.5,amount=0.004):
        image=self.img
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        coords=tuple(coords)
        out[coords] = 255

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        coords=tuple(coords)
        out[coords] = 0
        return out,self.box
    
    def gamma_trans(self,gamma=1.0):
        gamma_table=self.gamma_table
        if gamma!=1.0:
            gamma_table = [np.power(x, gamma)  for x in range(256)]
            gamma_table = np.array(gamma_table)
        src = cv2.LUT(self.img, gamma_table)
        cv2.normalize(src, src, 0, 255, cv2.NORM_MINMAX)
        return src,self.box
    
    def log_trans(self,log=1.0):
        log_table=self.log_table
        if log!=1.0:
            log_table = [np.log(1 + x) * log for x in range(256)]
            log_table = np.array(log_table)
        src = cv2.LUT(self.img,log_table)
        cv2.normalize(src, src, 0, 255, cv2.NORM_MINMAX)
        return src,self.box

In [4]:
data_aug=data_augmentation(imgsrc,box,gamma=2.0)

In [5]:
dstimg,dstbox=data_aug.resize()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_resize.jpg',dstimg)    

True

In [6]:
dstimg,dstbox=data_aug.crop()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_crop.jpg',dstimg)   

True

In [7]:
dstimg,dstbox=data_aug.lr_filp()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_lr_filp.jpg',dstimg)  

True

In [8]:
dstimg,dstbox=data_aug.up_filp()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_up_filp.jpg',dstimg)  


True

In [9]:
dstimg,dstbox=data_aug.hsv() 
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_hsv.jpg',dstimg)  


[[126 379 235 479]
 [121 163 251 295]]


In [10]:
dstimg,dstbox=data_aug.blur()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_blur.jpg',dstimg)  


True

In [11]:
dstimg,dstbox=data_aug.frosted_glass()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_frosted_glass.jpg',dstimg)  

True

In [12]:
dstimg,dstbox=data_aug.gamma_trans()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_gamma_trans.jpg',dstimg)  

True

In [13]:
dstimg,dstbox=data_aug.log_trans()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_log_trans.jpg',dstimg)  

True

In [14]:
dstimg,dstbox=data_aug.guass_noisy()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_guass_noisy.jpg',dstimg)  

True

In [15]:
dstimg,dstbox=data_aug.sp_noisy()
for box_id in range(len(dstbox)):
    dstimg=cv2.rectangle(dstimg,(int(dstbox[box_id][0]),int(dstbox[box_id][1])),(int(dstbox[box_id][2]),int(dstbox[box_id][3])),(0,255,0),2)
cv2.imwrite('img_sp_noisy.jpg',dstimg)  

True

In [16]:
#mix_up
imgadd=cv2.imread('/home/lishuang/Disk/pytorch/test/img/image_2017_04_20_01_37.jpg')

treeadd = ET.parse('/home/lishuang/Disk/pytorch/test/xml/image_2017_04_20_01_37.xml')
rootadd = treeadd.getroot()

print(imgadd.shape)  #H W C
objsadd=rootadd.findall('object')
boxadd=[]
for obj in objsadd:
    name=obj.find('name').text  #type
    bndbox=obj.find('bndbox')
    xmin=int(bndbox.find('xmin').text)
    ymin=int(bndbox.find('ymin').text)
    xmax=int(bndbox.find('xmax').text)
    ymax=int(bndbox.find('ymax').text)
    boxadd.append([xmin,ymin,xmax,ymax])
    

boxadd=np.array(boxadd)

imgout,boxout=data_aug.mix_up(imgadd,boxadd)

for box_id in range(len(boxout)):
    imgout=cv2.rectangle(imgout,(int(boxout[box_id][0]),int(boxout[box_id][1])),(int(boxout[box_id][2]),int(boxout[box_id][3])),(0,255,0),2)
cv2.imwrite('img_mixup.jpg',imgout)    
 

(480, 640, 3)


True